In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from snap_fit.puzzle import PuzzleConfig
from snap_fit.puzzle import PuzzleGenerator
from snap_fit.puzzle import PuzzleRasterizer
from snap_fit.puzzle import generate_label

## Test Label Generation

Verify the LLNN format label generation.


In [ ]:
# Test label generation
test_cases = [
    # (col, row, letter_digits, number_digits, expected)
    (0, 0, 1, 1, "A1"),
    (4, 4, 1, 1, "E5"),
    (0, 0, 1, 2, "A01"),
    (9, 9, 1, 2, "J10"),
    (25, 0, 1, 2, "Z01"),
    (26, 0, 2, 2, "BA01"),  # AA=0, AB=1, ..., AZ=25, BA=26
    (0, 0, 2, 2, "AA01"),
    (27, 29, 2, 2, "BB30"),
]

for col, row, ld, nd, expected in test_cases:
    result = generate_label(col, row, ld, nd)
    status = "✓" if result == expected else "✗"
    print(f"{status} col={col}, row={row} -> {result} (expected {expected})")

## Test PuzzleConfig


In [ ]:
# Test different puzzle sizes
configs = [
    PuzzleConfig(tiles_x=5, tiles_y=5),
    PuzzleConfig(tiles_x=10, tiles_y=10),
    PuzzleConfig(tiles_x=26, tiles_y=26),
    PuzzleConfig(tiles_x=30, tiles_y=30),
]

for cfg in configs:
    print(
        f"{cfg.tiles_x}x{cfg.tiles_y}: letter_digits={cfg.letter_digits}, number_digits={cfg.number_digits}"
    )
    print(f"  piece size: {cfg.piece_width:.1f}x{cfg.piece_height:.1f} mm")
    print(f"  auto font size: {cfg.auto_font_size:.2f} mm")
    print()

## Generate a Small Puzzle


In [ ]:
# Create a small puzzle
config = PuzzleConfig(
    width=100,
    height=80,
    tiles_x=5,
    tiles_y=4,
    seed=42,
    tab_size=0.2,
    jitter=0.04,
)

generator = PuzzleGenerator(config)
pieces = generator.generate()

print(f"Generated {len(pieces)} pieces")
print(f"Piece labels: {[p.label for p in pieces[:10]]}...")

## View SVG Output


In [ ]:
# Generate SVG
svg = generator.to_svg(include_labels=True)

# Save to file for viewing
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

svg_path = output_dir / "puzzle.svg"
svg_path.write_text(svg)
print(f"Saved SVG to {svg_path}")
print(f"SVG size: {len(svg)} bytes")

In [ ]:
# Display SVG inline
from IPython.display import SVG
from IPython.display import display

display(SVG(svg))

## Test Rasterization


In [ ]:
# Rasterize the puzzle
rasterizer = PuzzleRasterizer(dpi=150)  # Lower DPI for faster testing
img = rasterizer.rasterize(svg)

print(f"Image shape: {img.shape}")
print(f"Image dtype: {img.dtype}")

# Display
plt.figure(figsize=(12, 10))
plt.imshow(img[:, :, ::-1])  # BGR to RGB
plt.axis("off")
plt.title("Rasterized Puzzle")
plt.show()

## Test Individual Piece SVG


In [ ]:
# Get a piece with tabs on all sides (center piece)
piece_svg = generator.piece_to_svg(1, 2, include_label=True)

display(SVG(piece_svg))

In [ ]:
# Rasterize individual piece
piece_img = rasterizer.rasterize(piece_svg)

plt.figure(figsize=(6, 6))
plt.imshow(piece_img[:, :, ::-1])
plt.axis("off")
plt.title("Individual Piece (C2)")
plt.show()

## Test Corner and Edge Pieces


In [ ]:
# Show corner and edge pieces
positions = [
    (0, 0, "Top-Left Corner"),
    (0, 2, "Top Edge"),
    (0, 4, "Top-Right Corner"),
    (1, 0, "Left Edge"),
    (1, 2, "Center"),
    (1, 4, "Right Edge"),
    (3, 0, "Bottom-Left Corner"),
    (3, 2, "Bottom Edge"),
    (3, 4, "Bottom-Right Corner"),
]

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
axes = axes.flatten()

for ax, (row, col, title) in zip(axes, positions):
    piece_svg = generator.piece_to_svg(row, col, include_label=True)
    piece_img = rasterizer.rasterize(piece_svg)
    ax.imshow(piece_img[:, :, ::-1])
    ax.set_title(f"{title}\n{pieces[row * config.tiles_x + col].label}")
    ax.axis("off")

plt.tight_layout()
plt.show()

## Test Deterministic Generation

Same seed should produce identical puzzles.


In [ ]:
# Generate two puzzles with same seed
gen1 = PuzzleGenerator(PuzzleConfig(tiles_x=4, tiles_y=4, seed=123))
gen2 = PuzzleGenerator(PuzzleConfig(tiles_x=4, tiles_y=4, seed=123))

svg1 = gen1.to_svg()
svg2 = gen2.to_svg()

print(f"SVGs identical: {svg1 == svg2}")

In [ ]:
# Generate with different seed
gen3 = PuzzleGenerator(PuzzleConfig(tiles_x=4, tiles_y=4, seed=456))
svg3 = gen3.to_svg()

print(f"Different seed produces different SVG: {svg1 != svg3}")

## Verify Tab Shapes Match Reference

Compare with the JavaScript implementation output.


In [ ]:
# Create a puzzle matching the JS default settings
js_config = PuzzleConfig(
    width=300,
    height=200,
    tiles_x=15,
    tiles_y=10,
    tab_size=0.2,  # 20% in JS
    jitter=0.04,  # 4% in JS
    corner_radius=2.0,
    seed=42,
)

js_gen = PuzzleGenerator(js_config)
js_svg = js_gen.to_svg(include_labels=True)

# Save for comparison
js_svg_path = output_dir / "puzzle_js_like.svg"
js_svg_path.write_text(js_svg)

display(SVG(js_svg))

In [ ]:
print("Prototype complete!")
print(f"\nOutput files in: {output_dir.absolute()}")